In [ ]:
import os
import copy as cp
import numpy as np
import datetime as dt
from datetime import time, tzinfo, timedelta
from matplotlib import pyplot as plt

#%% custom importation
path_here = os.getcwd()
if path_here != os.path.join("/homedata", os.environ['USER'], "climaviation", "cheikh_code"):
    os.chdir(os.path.join("/homedata", os.environ['USER'], "climaviation", "cheikh_code"))
path_here = os.getcwd()

from image import readimage, pre_processing
from setup_variable import pathBuilder
from setup_variable.position import readLatLon
from calibration import worldToImage, convGeoCarto, reversZoom, projCoord, projCoord2

#%% function
def noName():
    return None

#%% image SIRTA

year = 2025#2023
month = 4 #3
day = 6 #20

for hour in np.arange(1, 16, 1): #hour = 12#10
    for minute in np.arange(00, 60, 2): #46

        dat = str(year) +'-'+str(month)+'-'+str(day)
        xmax = 901
        szamax = 60
        tps = time(hour, minute)
        path = pathBuilder()
        path.setAuto()
        path.setSite('SIRTA')
        # path.setSite('IPRAL')
        path.setDateDay(dt.datetime(year, month, day, hour, minute))

        imageName = path.getPathImage().split('/')[-1][:14]
        DirectoryImage = '/homedata/cdione/climaviation/skycamera/20250406/'#.join(path.getPathImage().split('/')[:-1])
        found, image = readimage(DirectoryImage = DirectoryImage, imageDate = imageName, 
                            imtype = '01', site = 'SIRTA', cle = False, verbose = [''])


        image1 = cp.deepcopy(image)
        image1[300, 400, :] = [0, 255, 0]

        imZoom     = pre_processing(image = image1, 
                                    processing = [{'name' : 'zoom', 'xmax' : xmax, 'szamax' : szamax}], 
                                    path = path, cle = False, verbose = [''])

        #%% test zoom and reverse zoom
        ix = 600
        iy = 400
        ixz, iyz = projCoord(ix, iy, image, szamax = 60, xmax = 901, zoom = True, imShow = True)
        ixNew, iyNew = projCoord(ixz, iyz, image, szamax = 60, xmax= 901, zoom = False, imShow = True)

        ix2 = 400
        iy2 = 600
        ixz2, iyz2 = projCoord(ix2, iy2, image, szamax = 60, xmax = 901, zoom = True, imShow = False)
        ixNew2, iyNew2 = projCoord(ixz2, iyz2, image, szamax = 60, xmax = 901, zoom = False, imShow = False)

        ix3 = 600
        iy3 = 400
        ixz3, iyz3 = projCoord2(ix3, iy3, szamax = 60, xmax = 901, zoom = True, imShow = True)
        ixNew3, iyNew3 = projCoord2(ixz3, iyz3, szamax = 60, xmax = 901, zoom = False, imShow = True)

        ix4 = 400
        iy4 = 600
        ixz4, iyz4 = projCoord2(ix4, iy4, szamax = 60, xmax = 901, zoom = True, imShow = False)
        ixNew4, iyNew4 = projCoord2(ixz4, iyz4, szamax = 60, xmax = 901, zoom = False, imShow = False)
                
        #%% point bretigny  
        lat_brety = 48.6 #° a corriger
        lon_brety = 2.3  #° à corriger.
        alt_avion = 11   #km à corriger
        radius=6370.0    #radius earth km

        #%% point IPRAL  
        # lat_ipral = 48.717823  #° a corriger
        lat_ipral = 48.701
        # lon_ipral = 2.207467 #° à corriger.
        lon_ipral = 2.23
        alt_avion = 11   #km à corriger
        radius = 6370.0    #radius earth km

        lat_site, lon_site = readLatLon('SIRTA')
        y_brety = radius*(lat_brety-lat_site)/180.*np.pi
        x_brety = radius*np.cos((lat_site+lat_brety)/2./180.*np.pi)*(lon_brety-lon_site)/180.*np.pi

        # lat_site, lon_site = readLatLon('IPRAL')
        y_ipral = radius*(lat_ipral-lat_site)/180.*np.pi
        x_ipral = radius*np.cos((lat_site+lat_ipral)/2./180.*np.pi)*(lon_ipral-lon_site)/180.*np.pi

        matPos = [-y_brety  /np.sqrt(x_brety**2+y_brety**2+alt_avion**2), 
                x_brety  /np.sqrt(x_brety**2+y_brety**2+alt_avion**2), 
                alt_avion/np.sqrt(x_brety**2+y_brety**2+alt_avion**2)]

        matPos1 = np.array(matPos)
        # print(matPos1)
        matPos_ipral = [-y_ipral  /np.sqrt(x_ipral**2+y_ipral**2+alt_avion**2), 
                x_ipral  /np.sqrt(x_ipral**2+y_ipral**2+alt_avion**2), 
                alt_avion/np.sqrt(x_ipral**2+y_ipral**2+alt_avion**2)]

        matPos1_ipral = np.array(matPos_ipral)

        ixs, iys = worldToImage(XPosition = matPos1, imageShape = image.shape, methodRead = "csv", site = 'SIRTA', zoom = False)
        print(matPos)
        print(ixs , iys)
        ixs_ipral, iys_ipral = worldToImage(XPosition = matPos1_ipral, imageShape = image.shape, methodRead = "csv", site = 'SIRTA', zoom = False)
        print(ixs_ipral, iys_ipral)
        ixs = ixs-68
        iys = iys-182
        xR, yR= reversZoom(ixs, iys, szamax= 60, xmax =901, xRGBc = 339, yRGBc= 339)

        ixs_ipral = ixs_ipral-68
        iys_ipral = iys_ipral-182
        
        ixs_brety = ixs-68
        iys_brety = iys-182
        xR_ipral, yR_ipral= reversZoom(ixs_ipral, iys_ipral, szamax= 60, xmax =901, xRGBc = 339, yRGBc= 339)

        ixsZoom, iysZoom = worldToImage(XPosition = matPos1, imageShape = imZoom.shape, methodRead = "csv", site = 'SIRTA', zoom = True)

        ixsZoom_ipral, iysZoom_ipral = worldToImage(XPosition = matPos1_ipral, imageShape = imZoom.shape, methodRead = "csv", site = 'SIRTA', zoom = True)
        ixsZoom_brety, iysZoom_brety = worldToImage(XPosition = matPos1, imageShape = imZoom.shape, methodRead = "csv", site = 'SIRTA', zoom = True)
        
        
        #%% plot

        fig1 = plt.figure(figsize=(10, 10))
        axes1 = fig1.add_subplot(1, 1, 1)
        axes1.imshow(image1[:,:, :])
        ixs, iys = projCoord(ixsZoom, iysZoom, image, szamax = 60, xmax= 901, zoom = False, imShow = True)
        ixsipral, iysipral = projCoord(ixsZoom_ipral, iysZoom_ipral, image, szamax = 60, xmax= 901, zoom = False, imShow = True)
        # axes1.scatter(ixs , iys , c= 'red', s = 2)
        axes1.scatter(ixs_ipral , iys_ipral , c= 'black', s = 2)
        axes1.scatter(ixs_brety , iys_brety , c= 'red', s = 2)
        print('lon =',ixs_ipral , 'lat =', iys_ipral)
        fig2 = plt.figure(figsize=(10, 10))
        axes2 = fig2.add_subplot(1, 1, 1)
        axes2.imshow(imZoom)

        # axes2.scatter(ixsZoom, iysZoom, c = 'red', s = 2)
        axes2.scatter(ixsZoom_ipral, iysZoom_ipral, c = 'black', s = 15)
        axes2.scatter(ixsZoom_brety, iysZoom_brety, c = 'black', s = 15)
        axes2.text(ixsZoom_ipral+10, iysZoom_ipral-10, 'IPRAL', c = 'black')
        axes2.text(ixsZoom_brety+10, iysZoom_brety-10, 'Bretigny', c = 'red')
        
        print('lon =', ixsZoom_ipral, 'lat =', iysZoom_ipral)
        axes2.set_xticklabels('')
        axes2.set_yticklabels('')
        axes2.set_title(str(time(hour, minute)))
        fig2.savefig('sky_camera_climaviation_'+dat+'-'+str(hour)+'_'+str(minute)+'.png', dpi = 150)
        plt.close()

not implemented
[np.float64(0.6974297124754048), np.float64(0.37508516928733177), np.float64(0.6106577698983021)]
554.6195613288221 620.5944108310935
408.8456062012428 552.9262248205251
lon = 340.8456062012428 lat = 370.92622482052514
lon = 452 lat = 489
not implemented
[np.float64(0.6974297124754048), np.float64(0.37508516928733177), np.float64(0.6106577698983021)]
554.6195613288221 620.5944108310935
408.8456062012428 552.9262248205251
lon = 340.8456062012428 lat = 370.92622482052514
lon = 452 lat = 489
not implemented
[np.float64(0.6974297124754048), np.float64(0.37508516928733177), np.float64(0.6106577698983021)]
554.6195613288221 620.5944108310935
408.8456062012428 552.9262248205251
lon = 340.8456062012428 lat = 370.92622482052514
lon = 452 lat = 489
not implemented
[np.float64(0.6974297124754048), np.float64(0.37508516928733177), np.float64(0.6106577698983021)]
554.6195613288221 620.5944108310935
408.8456062012428 552.9262248205251
lon = 340.8456062012428 lat = 370.92622482052514


/tmp/ipykernel_2805269/3947912009.py:143: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig2 = plt.figure(figsize=(10, 10))


not implemented
[np.float64(0.6974297124754048), np.float64(0.37508516928733177), np.float64(0.6106577698983021)]
554.6195613288221 620.5944108310935
408.8456062012428 552.9262248205251


/tmp/ipykernel_2805269/3947912009.py:134: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig1 = plt.figure(figsize=(10, 10))


lon = 340.8456062012428 lat = 370.92622482052514
lon = 452 lat = 489
not implemented
[np.float64(0.6974297124754048), np.float64(0.37508516928733177), np.float64(0.6106577698983021)]
554.6195613288221 620.5944108310935
408.8456062012428 552.9262248205251
lon = 340.8456062012428 lat = 370.92622482052514
lon = 452 lat = 489
not implemented
[np.float64(0.6974297124754048), np.float64(0.37508516928733177), np.float64(0.6106577698983021)]
554.6195613288221 620.5944108310935
408.8456062012428 552.9262248205251
lon = 340.8456062012428 lat = 370.92622482052514
lon = 452 lat = 489
not implemented
[np.float64(0.6974297124754048), np.float64(0.37508516928733177), np.float64(0.6106577698983021)]
554.6195613288221 620.5944108310935
408.8456062012428 552.9262248205251
lon = 340.8456062012428 lat = 370.92622482052514
lon = 452 lat = 489
not implemented
[np.float64(0.6974297124754048), np.float64(0.37508516928733177), np.float64(0.6106577698983021)]
554.6195613288221 620.5944108310935
408.845606201242